In [ ]:
import os
import numpy as np
import cv2

In [ ]:
ROOT_DIR = 'road_segmentation_ideal'

train_inputs_dir = os.path.join(ROOT_DIR, 'training/input')
train_masks_dir = os.path.join(ROOT_DIR, 'training/output')

test_inputs_dir = os.path.join(ROOT_DIR, 'testing/input')
test_masks_dir = os.path.join(ROOT_DIR, 'testing/output')

train_df = os.listdir(train_masks_dir)
test_df = os.listdir(test_inputs_dir)

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
class RoadsDataset(Dataset):
  
  def __init__(self, df, images_dir, masks_dir, augmentation=None, preprocessing=None):

    self.mask_names = df

    self.images = [os.path.join(images_dir, image_name) for image_name in self.mask_names]
    self.masks = [os.path.join(masks_dir, image_name) for image_name in self.mask_names]

    self.augmentation = augmentation
    self.preprocessing = preprocessing

  def __getitem__(self, i):

    image = cv2.imread(self.images[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.array(image)
    mask = cv2.imread(self.masks[i], 0)
    masks = [(mask == 255)]
    mask = np.stack(masks, axis=-1).astype('float')
    
    if self.augmentation:  
      aug=self.augmentation(image=image, mask=mask)
      image, mask = aug['image'], aug['mask']

    if self.preprocessing:
      sample = self.preprocessing(image=image, mask=mask)
      image, mask = sample['image'], sample['mask']

    return image, mask


  def __len__(self):
    return len(self.mask_names)

In [ ]:
def RoadsDSLoader(df, imgs_dir, masks_dir, batch_size, shuffle, num_workers, augmentation = None, preprocessing = None):

  dataset = RoadsDataset(df, imgs_dir, masks_dir, augmentation=augmentation, preprocessing=preprocessing)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

  return dataloader

In [ ]:
import albumentations as albu

In [ ]:
def get_train_aug():

    train_transform = [
        albu.HorizontalFlip(p=0.5),
        albu.RandomCrop(height=320, width=320, always_apply=True)
    ]
    return albu.Compose(train_transform)


def get_val_aug():

    val_transform = [
        albu.PadIfNeeded(min_height=1504, min_width=1504, border_mode = cv2.BORDER_CONSTANT, value=0, mask_value=0)
    ]
    return albu.Compose(val_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):

    transform_  = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor)
    ]
    return albu.Compose(transform_)

In [ ]:
import torch
import segmentation_models_pytorch as smp

In [ ]:
ENCODER = 'efficientnet-b5'
ENCODER_WEIGHTS = 'imagenet'
DEVICE = 'cuda'

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=1, 
    activation='sigmoid',
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth


  0%|          | 0.00/117M [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_valid = train_test_split(train_df, test_size=0.15, random_state = 42)

train_loader = RoadsDSLoader(df_train, train_inputs_dir, train_masks_dir, batch_size=16, shuffle=True, num_workers=1, augmentation=get_train_aug(), preprocessing=get_preprocessing(preprocessing_fn))
valid_loader = RoadsDSLoader(df_valid, train_inputs_dir, train_masks_dir, batch_size=2, shuffle=False, num_workers=1, augmentation=get_val_aug(), preprocessing=get_preprocessing(preprocessing_fn))

In [ ]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold = 0.5),
    smp.utils.metrics.Accuracy(),
    smp.utils.metrics.Precision(),
    smp.utils.metrics.Recall()
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [15]:
max_score = 0

for i in range(40):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, 'best_model.pth')
        
    if i == 30:
        optimizer.param_groups[0]['lr'] = 1e-5


Epoch: 0
valid: 100%|██████████| 61/61 [02:53<00:00,  2.84s/it, dice_loss - 0.8448, iou_score - 0.1553, accuracy - 0.7749, precision - 0.1612, recall - 0.8178]

Epoch: 1
valid: 100%|██████████| 61/61 [00:31<00:00,  1.95it/s, dice_loss - 0.7733, iou_score - 0.3115, accuracy - 0.9043, precision - 0.3332, recall - 0.8273]

Epoch: 2
valid: 100%|██████████| 61/61 [00:31<00:00,  1.94it/s, dice_loss - 0.7294, iou_score - 0.3327, accuracy - 0.9101, precision - 0.35, recall - 0.8691]

Epoch: 3
valid: 100%|██████████| 61/61 [00:32<00:00,  1.88it/s, dice_loss - 0.6908, iou_score - 0.4, accuracy - 0.9336, precision - 0.4316, recall - 0.845]

Epoch: 4
valid: 100%|██████████| 61/61 [00:32<00:00,  1.86it/s, dice_loss - 0.6513, iou_score - 0.3887, accuracy - 0.9279, precision - 0.4137, recall - 0.8651]

Epoch: 5
valid: 100%|██████████| 61/61 [00:31<00:00,  1.92it/s, dice_loss - 0.6001, iou_score - 0.4344, accuracy - 0.9412, precision - 0.4702, recall - 0.8514]

Epoch: 6
valid: 100%|██████████| 61/61 

In [16]:
best_model = torch.load('best_model.pth')

In [17]:
test_epoch = smp.utils.train.ValidEpoch(
    model=best_model,
    loss=loss,
    metrics=metrics,
    device=DEVICE,
)

In [18]:
test_loader = RoadsDSLoader(test_df, test_inputs_dir, test_masks_dir, batch_size=4, shuffle=False, num_workers=1, augmentation=get_val_aug(), preprocessing=get_preprocessing(preprocessing_fn))

logs = test_epoch.run(test_loader)

valid: 100%|██████████| 4/4 [00:22<00:00,  5.68s/it, dice_loss - 0.292, iou_score - 0.5633, accuracy - 0.9664, precision - 0.7532, recall - 0.6915]


In [19]:
test_dataset = RoadsDataset(test_df, test_inputs_dir, test_masks_dir, augmentation=get_val_aug(), preprocessing=get_preprocessing(preprocessing_fn))

In [20]:
for i in range(len(test_df)):
    img_name = test_df[i]
    image, gt_mask = test_dataset[i]
    
    gt_mask = gt_mask.squeeze()
    
    x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask = pr_mask.squeeze().cpu()
    cv2.imwrite('preds/' + img_name, 255 * np.array(pr_mask))